# Zurich Real Estate Price Prediction - Model Development

This notebook focuses on building and evaluating machine learning models to predict real estate prices in Zurich based on property characteristics and travel time.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import pickle
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## Load Processed Data

We'll load the processed datasets that were created during data preparation.

In [ ]:
# Function to load processed data files
def load_processed_data():
    processed_dir = os.path.join('..', 'data', 'processed')
    
    # Check if processed files exist
    neighborhood_path = os.path.join(processed_dir, 'processed_neighborhood_data.csv')
    building_age_path = os.path.join(processed_dir, 'processed_building_age_data.csv')
    travel_times_path = os.path.join(processed_dir, 'travel_times.json')
    
    if not os.path.exists(neighborhood_path) or not os.path.exists(building_age_path):
        print("Processed data files not found. Please run data_preparation.py first.")
        return None, None, None
    
    # Load the data
    neighborhood_df = pd.read_csv(neighborhood_path)
    building_age_df = pd.read_csv(building_age_path)
    
    # Check if travel times file exists
    if os.path.exists(travel_times_path):
        with open(travel_times_path, 'r') as f:
            travel_times = json.load(f)
    else:
        print("Travel times data not found. Using None for now.")
        travel_times = None
    
    return neighborhood_df, building_age_df, travel_times

# Load data
neighborhood_df, building_age_df, travel_times = load_processed_data()

if neighborhood_df is not None:
    print(f"Loaded {len(neighborhood_df)} neighborhood records")
    print(f"Loaded {len(building_age_df)} building age records")
    print(f"Travel times data available: {travel_times is not None}")
    
    # Display the first few rows of each dataset
    neighborhood_df.head()

In [ ]:
building_age_df.head()

## Prepare Data for Modeling

Let's prepare our feature set for modeling by combining neighborhood and building age data, and optionally adding travel time features.

In [ ]:
def prepare_model_data(neighborhood_df, building_age_df, travel_times=None, latest_year_only=True):
    """Prepare data for model training"""
    if neighborhood_df is None or building_age_df is None:
        return None, None
    
    # Filter for the latest year if requested
    if latest_year_only:
        latest_year = max(neighborhood_df['year'].max(), building_age_df['year'].max())
        n_df = neighborhood_df[neighborhood_df['year'] == latest_year].copy()
        ba_df = building_age_df[building_age_df['year'] == latest_year].copy()
    else:
        n_df = neighborhood_df.copy()
        ba_df = building_age_df.copy()
    
    # For now, we'll use the neighborhood data as our primary dataset
    # In a real implementation, we would join these datasets properly
    X = n_df[['neighborhood', 'room_count', 'year']]
    y = n_df['median_price']
    
    # If travel times are available, we could add them as features
    # This is a placeholder for demonstration
    if travel_times is not None:
        print("Adding travel time features...")
        
        # Example: Add average travel time to key destinations as a feature
        # In a real implementation, this would be done more carefully
        X['avg_travel_time'] = X['neighborhood'].apply(
            lambda x: np.mean(list(travel_times.get(x, {}).values())) if x in travel_times else np.nan
        )
        
        # Fill missing values with the mean
        X['avg_travel_time'].fillna(X['avg_travel_time'].mean(), inplace=True)
    
    return X, y

# Prepare data for modeling
X, y = prepare_model_data(neighborhood_df, building_age_df, travel_times, latest_year_only=True)

if X is not None:
    print(f"Prepared {len(X)} samples for modeling")
    print(f"Feature columns: {X.columns.tolist()}")
    X.head()

## Train-Test Split

Let's split our data into training and testing sets.

In [ ]:
if X is not None and len(X) > 0:
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"Training set: {len(X_train)} samples")
    print(f"Testing set: {len(X_test)} samples")

## Define Preprocessing Pipeline

We'll create a preprocessing pipeline to handle categorical features.

In [ ]:
def create_preprocessor(X):
    """Create a preprocessing pipeline for model features"""
    # Identify categorical and numerical columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Create preprocessing for categorical features (one-hot encoding)
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Create preprocessing for numerical features (scaling)
    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_cols),
            ('num', numerical_transformer, numerical_cols)
        ]
    )
    
    return preprocessor

if X is not None and len(X) > 0:
    # Create preprocessor
    preprocessor = create_preprocessor(X)
    print("Preprocessing pipeline created")

## Baseline Model: Random Forest

Let's start with a Random Forest regression model as our baseline.

In [ ]:
if X is not None and len(X) > 0:
    # Create pipeline with preprocessing and Random Forest
    rf_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(
            n_estimators=100,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            random_state=42
        ))
    ])
    
    # Train the model
    print("Training Random Forest model...")
    rf_pipeline.fit(X_train, y_train)
    
    # Evaluate on training data
    y_train_pred = rf_pipeline.predict(X_train)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    
    print(f"Training set evaluation:")
    print(f"MAE: {train_mae:.2f} CHF")
    print(f"RMSE: {train_rmse:.2f} CHF")
    print(f"R²: {train_r2:.4f}")
    
    # Evaluate on testing data
    y_test_pred = rf_pipeline.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_r2 = r2_score(y_test, y_test_pred)
    
    print(f"\nTesting set evaluation:")
    print(f"MAE: {test_mae:.2f} CHF")
    print(f"RMSE: {test_rmse:.2f} CHF")
    print(f"R²: {test_r2:.4f}")

## Alternative Model: Gradient Boosting

Let's try a Gradient Boosting model and compare its performance.

In [ ]:
if X is not None and len(X) > 0:
    # Create pipeline with preprocessing and Gradient Boosting
    gb_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            random_state=42
        ))
    ])
    
    # Train the model
    print("Training Gradient Boosting model...")
    gb_pipeline.fit(X_train, y_train)
    
    # Evaluate on testing data
    y_test_pred_gb = gb_pipeline.predict(X_test)
    test_mae_gb = mean_absolute_error(y_test, y_test_pred_gb)
    test_rmse_gb = np.sqrt(mean_squared_error(y_test, y_test_pred_gb))
    test_r2_gb = r2_score(y_test, y_test_pred_gb)
    
    print(f"Testing set evaluation:")
    print(f"MAE: {test_mae_gb:.2f} CHF")
    print(f"RMSE: {test_rmse_gb:.2f} CHF")
    print(f"R²: {test_r2_gb:.4f}")

## Feature Importance Analysis

Let's examine which features are most important for predicting property prices.

In [ ]:
def plot_feature_importances(pipeline, X):
    """Plot feature importances from a trained Pipeline with a tree-based model"""
    if hasattr(pipeline[-1], 'feature_importances_'):
        # Get the preprocessor
        preprocessor = pipeline.named_steps['preprocessor']
        
        # Get feature names after preprocessing
        feature_names = []
        
        # Get categorical column names after one-hot encoding
        cat_cols = X.select_dtypes(include=['object', 'category']).columns
        if len(cat_cols) > 0:
            ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
            cat_features = ohe.get_feature_names_out(cat_cols)
            feature_names.extend(cat_features)
        
        # Get numerical column names
        num_cols = X.select_dtypes(include=['int64', 'float64']).columns
        feature_names.extend(num_cols)
        
        # Get feature importances
        importances = pipeline[-1].feature_importances_
        
        # Create a DataFrame for easier plotting
        feature_importances = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        }).sort_values('Importance', ascending=False)
        
        # Plot the top N features
        top_n = min(20, len(feature_importances))
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importances.head(top_n))
        plt.title(f'Top {top_n} Feature Importances')
        plt.tight_layout()
        plt.show()
        
        return feature_importances
    else:
        print("Model does not have feature_importances_ attribute")
        return None

if X is not None and len(X) > 0:
    # Plot feature importances for Random Forest
    print("Random Forest Feature Importances:")
    rf_feature_importances = plot_feature_importances(rf_pipeline, X)

In [ ]:
if X is not None and len(X) > 0:
    # Plot feature importances for Gradient Boosting
    print("Gradient Boosting Feature Importances:")
    gb_feature_importances = plot_feature_importances(gb_pipeline, X)

## Model Hyperparameter Tuning

Let's use grid search to find the best hyperparameters for our model.

In [ ]:
if X is not None and len(X) > 0 and len(X) > 100:  # Only run if we have enough data
    # Define parameter grid for Random Forest
    param_grid = {
        'model__n_estimators': [50, 100],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5, 10]
    }
    
    # Create grid search
    print("Running Grid Search for Random Forest (this may take a while)...")
    grid_search = GridSearchCV(
        rf_pipeline,
        param_grid,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    
    # Fit grid search
    grid_search.fit(X, y)
    
    # Print best parameters
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best RMSE: {np.sqrt(-grid_search.best_score_):.2f} CHF")
    
    # Create final model with best parameters
    best_rf_model = grid_search.best_estimator_

## Save the Final Model

Let's save the best model for use in our Streamlit app.

In [ ]:
def save_model(model, model_name='price_model'):
    """Save the trained model to disk"""
    # Create models directory if it doesn't exist
    models_dir = os.path.join('..', 'models')
    os.makedirs(models_dir, exist_ok=True)
    
    # Save the model
    model_path = os.path.join(models_dir, f'{model_name}.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"Model saved to {model_path}")
    return model_path

if X is not None and len(X) > 0:
    # Choose the best model to save
    if 'best_rf_model' in locals():
        final_model = best_rf_model
        print("Saving the best Random Forest model from grid search...")
    elif 'rf_pipeline' in locals() and 'gb_pipeline' in locals():
        # Compare RF and GB models
        if test_r2 >= test_r2_gb:
            final_model = rf_pipeline
            print("Random Forest performed better. Saving this model...")
        else:
            final_model = gb_pipeline
            print("Gradient Boosting performed better. Saving this model...")
    else:
        final_model = rf_pipeline
        print("Saving the Random Forest model...")
    
    # Save the model
    model_path = save_model(final_model)

## Model Testing

Let's test our model with some sample inputs to make sure it works as expected.

In [ ]:
if X is not None and len(X) > 0:
    # Create some sample inputs
    neighborhoods = X['neighborhood'].unique()[:3]  # Take first 3 neighborhoods
    room_counts = X['room_count'].unique()[:2]  # Take first 2 room counts
    
    print("Testing model predictions for sample inputs:")
    print("\nNeighborhoods:", neighborhoods)
    print("Room counts:", room_counts)
    
    for neighborhood in neighborhoods:
        for room_count in room_counts:
            # Create input features
            sample_input = pd.DataFrame({
                'neighborhood': [neighborhood],
                'room_count': [room_count],
                'year': [X['year'].iloc[0]]  # Use the same year as in the dataset
            })
            
            # Add travel time if it was included in the model
            if 'avg_travel_time' in X.columns:
                avg_time = travel_times.get(neighborhood, {})
                if avg_time:
                    sample_input['avg_travel_time'] = np.mean(list(avg_time.values()))
                else:
                    sample_input['avg_travel_time'] = X['avg_travel_time'].mean()
            
            # Make prediction
            prediction = final_model.predict(sample_input)[0]
            
            print(f"\nPrediction for {room_count} room property in {neighborhood}:")
            print(f"Estimated price: {prediction:,.2f} CHF")

## Final Model Performance Summary

Let's summarize the performance of our final model.

In [ ]:
if X is not None and len(X) > 0:
    # Print final model summary
    print("Final Model Performance Summary:")
    
    # Re-evaluate on test set
    y_pred_final = final_model.predict(X_test)
    final_mae = mean_absolute_error(y_test, y_pred_final)
    final_rmse = np.sqrt(mean_squared_error(y_test, y_pred_final))
    final_r2 = r2_score(y_test, y_pred_final)
    
    print(f"\nTest Set Performance:")
    print(f"Mean Absolute Error: {final_mae:,.2f} CHF")
    print(f"Root Mean Squared Error: {final_rmse:,.2f} CHF")
    print(f"R² Score: {final_r2:.4f}")
    
    # Visualize actual vs predicted prices
    plt.figure(figsize=(10, 8))
    plt.scatter(y_test, y_pred_final, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Actual Price (CHF)')
    plt.ylabel('Predicted Price (CHF)')
    plt.title('Actual vs Predicted Real Estate Prices')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## Conclusion

In this notebook, we've developed a machine learning model to predict real estate prices in Zurich based on property characteristics. Key findings:

1. Model Performance: [To be filled after running]
2. Key Features: [To be filled after running]
3. Patterns by Neighborhood: [To be filled after running]
4. Price Sensitivity to Room Count: [To be filled after running]

The final model has been saved and can be used in our Streamlit application for interactive price prediction.